In [1]:
# -*- coding: utf-8 -*-
######################### IMPORTAR LIBRERÍAS #########################
import os
from dotenv import load_dotenv, find_dotenv
from pymilvus import connections, db, CollectionSchema, FieldSchema, DataType, Collection, MilvusClient

# import Langchain modules
from langchain.agents import tool
from langchain_text_splitters import CharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.documents import Document
from langchain_community.document_loaders import TextLoader, PyPDFDirectoryLoader
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_milvus.vectorstores import Milvus

######################### VARIABLES DE ENTORNO #########################
QUERY = "¿Qué es el Parkinson?"

LLM_MODEL = "gpt-4o-mini"
EMBEDDING_MODEL = "text-embedding-3-large"
DIMENSION_EMBEDDING = 3072

INPUT_FILE = "parkinson_gpt.txt"
BD_NAME = "Joaquin_DB90"
COLLECTION_NAME = "Joaquin_Col"

URI_CONNECTION = "http://localhost:19530"
HOST = "localhost"
PORT = 19530

######################### OBTENER API KEY DE OPENAI #########################
# Carga las variables de entorno desde un archivo .env
load_dotenv(find_dotenv(), override=True)

# Obtiene la API key de OpenAI desde las variables de entorno
api_key_openAI = os.environ.get("OPENAI_API_KEY")
print(api_key_openAI)

sk-proj-bLpI4ExJAbHOa2n9R3DaT3BlbkFJBI6fQ7K2ExHIlj8EJPUk


In [2]:

######################### CARGAR EL DOCUMENTO #########################

# load the document and split it into chunks

# loader = PyPDFDirectoryLoader("./folder")
loader = TextLoader(INPUT_FILE)
documents = loader.load()

# split it into chunks
text_splitter = CharacterTextSplitter(
    separator="\n\n", 
    chunk_overlap=0
    )
docs = text_splitter.split_documents(documents)

######################## ADDING METADATA TO DOCUMENTS ########################
# Adding metadata to documents
for i, doc in enumerate(docs):
    doc.metadata["chunk"] = i
    doc.metadata["Title"] = "Parkinson"
    doc.metadata["Capitulo"] = "30"
    doc.metadata["author"] = "Joaquin Chamorro"

In [3]:
print(documents)

[Document(metadata={'source': 'parkinson_gpt.txt'}, page_content='# Trastornos del Movimiento\n\nBlair Ford, MD  \nHoward Geyer, MD, PhD  \nSusan B. Bressman, MD\n\nLos trastornos del movimiento son condiciones que producen ya sea una reducción o un exceso de movimiento. Los trastornos neurológicos que resultan en una escasez o lentitud de movimiento se denominan trastornos hipocinéticos. La categoría de trastornos hipocinéticos está representada por la enfermedad de Parkinson y otras causas de parkinsonismo. Los trastornos hipercinéticos se caracterizan por movimientos excesivos e involuntarios. Los trastornos hipercinéticos generalmente pueden clasificarse en una de las cinco categorías principales de movimientos anormales: distonía, corea, temblor, mioclonía o tics.\n\nLos movimientos anormales pueden ser difíciles de reconocer o categorizar debido a su apariencia inusual, complejidad, sutileza o variabilidad. Los especialistas en trastornos del movimiento tienden a aislar o reducir

In [17]:

######################### CONECTAR A MILVUS #########################

uri = URI_CONNECTION

client = MilvusClient(
    uri=uri,
    token="joaquin:chamorro"
)

connections.connect(alias="default", host=HOST, port=PORT)

######################### CREAR LA BASE DE DATOS EN MILVUS #########################

db_name = BD_NAME
if db_name not in db.list_database():
    db.create_database(db_name, using="default")
    db.using_database(db_name, using="default")
else:
    db.using_database(db_name, using="default")

print(f"Conectado a la base de datos {db_name}")

Conectado a la base de datos Joaquin_DB90


In [45]:

######################### CREAR LA COLECCIÓN EN MILVUS #########################
collection_name = COLLECTION_NAME
dimension = DIMENSION_EMBEDDING  # Especifica la dimensión del vector

######################### GUARDAR LOS VECTORES EN MILVUS - VECTORSTORE #########################
#vector_store = Milvus()

# Crear la función de embeddings de código abierto
embeddings = OpenAIEmbeddings(model="text-embedding-3-large", api_key=api_key_openAI)

index_params = {
    "index_type": "HNSW",
    "metric_type": "COSINE",
    "params": {"M": 8, "efConstruction": 64}
}

if collection_name not in client.list_collections():
    vector_store = Milvus.from_documents(
        docs,
        embedding=embeddings,
        collection_name=collection_name,
        connection_args={"uri": uri},
        primary_field="pk",
        text_field="text",
        vector_field="vector",
        index_params=index_params,
        enable_dynamic_field=True,
        drop_old=True
    )
    print("Colección creada")
else:
    vector_store = Milvus(
        embeddings,
        collection_name=collection_name,
        connection_args={"uri": uri},
        enable_dynamic_field=True,
        primary_field="pk",
        text_field="text",
        vector_field="vector",
        index_params=index_params
    )
    print("Colección ya existe")

print(f"Conexión a Milvus-VectorStore establecida.\nConectado a colleccion: {collection_name}\n")



KeyError: 'FLAT'

In [44]:
print(vector_store._get_index())
print(vector_store._get_index().get("collection"))

{'collection': 'Joaquin_Col', 'field': 'vector', 'index_name': 'vector', 'index_param': {'index_type': 'HNSW', 'metric_type': 'COSINE', 'params': {'M': 8, 'efConstruction': 64}}}
Joaquin_Col


In [7]:

######################### RECUPERAR LOS VECTORES SIMILARES - RETRIEVER #########################

query = QUERY

model = ChatOpenAI(api_key=api_key_openAI, model=LLM_MODEL)

retriever = vector_store.as_retriever(
    search_type="similarity", search_kwargs={"k": 10, "filter": {"chapter": "30"}})
    #search_type="mmr", search_kwargs={"k": 10, "fetch_k": 50})



In [8]:

######################### EJECUTAR EL PIPELINE #########################

template =  """
            - Contesta como un profesional medico: {context}
            - Si no se aportan documentos:
                - Menciona que no se aportan documentos
                - Responde con tu conocimiento
            - Question: {question}
            """
prompt = ChatPromptTemplate.from_template(template)
output_parser = StrOutputParser()
setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()})
chain = setup_and_retrieval | prompt | model | output_parser
respuesta=chain.invoke(query)

print(respuesta)



La enfermedad de Parkinson (EP) es un trastorno neurodegenerativo progresivo que afecta principalmente el control motor. Se caracteriza por la degeneración de las neuronas dopaminérgicas en la sustancia negra del cerebro, lo que da lugar a una disminución de la dopamina, un neurotransmisor crucial para el control del movimiento. Los síntomas motores más típicos de la EP incluyen:

1. **Temblor en reposo**: Este es un movimiento involuntario, oscilante, que generalmente comienza en una extremidad (a menudo una mano) y puede aumentar con el estrés.
2. **Bradicinesia**: Se refiere a la lentitud en la iniciación y ejecución de movimientos, lo que puede dificultar actividades cotidianas.
3. **Rigidez**: Los músculos se tensan, lo que puede causar dolor y limitar el rango de movimiento.
4. **Pérdida de reflejos posturales**: Esto puede resultar en inestabilidad y caídas.
5. **Postura flexionada**: Los pacientes a menudo tienen una postura encorvada.
6. **Congelación**: Sensación de que el cu